In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 성남시 거주 인구
population = pd.read_csv('/content/1.성남시_거주인구.csv')
# 성남시 상권 정보
market_info = pd.read_csv('/content/2.성남시_상권정보.csv')
# 성남시 상가개폐업
market_open = pd.read_csv('/content/3.성남시_상가개폐업.csv')
# 성남시 개별 공시지가
seperate_cost = pd.read_csv('/content/8.성남시_개별공시지가.csv')

# 1. 거주인구
- gid   
  - 100MX100M 격자 ID
- year    
  - 기준연도
- m_20g_pop    
  - 20세이상 29세이하 남성 인구 수
- w_20g_pop    
  - 20세이상 29세이하 여성 인구 수
- m_30g_pop    
  - 30세이상 39세이하 남성 인구 수
- w_30g_pop    
  - 30세이상 39세이하 여성 인구 수
- m_40g_pop
  - 40세이상 49세이하 남성 인구 수
- w_40g_pop
  - 40세이상 49세이하 여성 인구 수
- m_50g_pop
  - 50세이상 59세이하 남성 인구 수
- w_50g_pop
  - 50세이상 59세이하 여성 인구 수
- m_60g_pop
  - 60세이상 69세이하 남성 인구 수
- w_60g_pop
  - 60세이상 69세이하 여성 인구 수
- m_70g_pop
  - 70세이상 79세이하 남성 인구 수
- w_70g_pop
  - 70세이상 79세이하 여성 인구 수
- m_80g_pop
  - 80세이상 89세이하 남성 인구 수
- w_80g_pop
  - 80세이상 89세이하 여성 인구 수
- m_90g_pop
  - 90세이상 99세이하 남성 인구 수
- w_90g_pop
  - 90세이상 99세이하 여성 인구 수
- m_100g_pop
  - 100세이상 남성 인구 수
- w_100g_pop
  - 100세이상 여성 인구 수

In [ ]:
population.head()

In [ ]:
population.describe()

In [ ]:
# 결측값 처리: NaN을 0으로 대체
population_cleaned = population.fillna(0)

# 결측값이 제거된 데이터 확인
print("==== 결측값 처리 후 ====")
print(population_cleaned.info())

- 결측값을 0으로 대체하는게 맞는가?
- null값이 무엇을 의미하는가?
- null이랑 0이랑 다른것같은데
- 근데 null값이 있는 행을 전부 제거하면 31개밖에 남지 않는다

In [ ]:
# 연도별 총인구 데이터 계산 (만약 total_population 열이 없을 경우)
if 'total_population' not in population_cleaned.columns:
    # 연령대별 열 이름 필터링
    age_columns = [col for col in population_cleaned.columns if '_pop' in col]
    # 총인구 계산 및 열 추가
    population_cleaned['total_population'] = population_cleaned[age_columns].sum(axis=1)

# 연도별 총인구 데이터 준비
yearly_population = population_cleaned.groupby('year')['total_population'].sum().reset_index()

# Seaborn 스타일 설정
sns.set_theme(style="whitegrid")

# Seaborn Barplot
plt.figure(figsize=(10, 6))
sns.barplot(data=yearly_population, x='year', y='total_population', palette="Blues_d")

# 스타일 및 레이블 설정
plt.title("Yearly Population", fontsize=16)
plt.xlabel("Year", fontsize=12)
plt.ylabel("Total Population", fontsize=12)
plt.xticks(fontsize=10, rotation=45)  # X축 레이블 회전
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


**- 연도별로 큰 변화 없음**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 남성 및 여성 총인구 계산
male_columns = [col for col in population_cleaned.columns if col.startswith('m_')]
female_columns = [col for col in population_cleaned.columns if col.startswith('w_')]

# 연도별 남성 및 여성 총합 계산
yearly_gender_population = population_cleaned.groupby('year')[
    male_columns + female_columns
].sum()
yearly_gender_population['male_total'] = yearly_gender_population[male_columns].sum(axis=1)
yearly_gender_population['female_total'] = yearly_gender_population[female_columns].sum(axis=1)

# 데이터 정리
yearly_gender_population = yearly_gender_population[['male_total', 'female_total']].reset_index()
tidy_data = yearly_gender_population.melt(id_vars='year', var_name='Gender', value_name='Population')

# Seaborn Barplot with hue
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.barplot(data=tidy_data, x='year', y='Population', hue='Gender', palette="coolwarm")

# 스타일링
plt.title("Yearly Population by Gender", fontsize=16)
plt.xlabel("Year", fontsize=12)
plt.ylabel("Total Population", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(title="Gender", fontsize=10)
plt.tight_layout()
plt.show()


- 남녀 인구 비율도 비슷

In [ ]:
# 데이터 준비
age_groups = ["20g", "30g", "40g", "50g", "60g", "70g", "80g", "90g", "100g"]
male_columns = [f"m_{age}_pop" for age in age_groups]
female_columns = [f"w_{age}_pop" for age in age_groups]

male_totals = population_cleaned[male_columns].sum()
female_totals = population_cleaned[female_columns].sum()

age_data = pd.DataFrame({
    "Age Group": age_groups * 2,
    "Population": list(male_totals) + list(female_totals),
    "Gender": ["Male"] * len(age_groups) + ["Female"] * len(age_groups)
})


sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.barplot(data=age_data, x="Age Group", y="Population", hue="Gender", palette="coolwarm")

# 스타일 및 레이블 추가
plt.title("Population Distribution by Age Group and Gender", fontsize=16)
plt.xlabel("Age Groups", fontsize=12)
plt.ylabel("Population", fontsize=12)
plt.legend(title="Gender", fontsize=10)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


**- 40~60대 인구가 가장 많고, 고령층(70대 이상)은 상대적으로 적음**


In [ ]:
# 지역별 총인구 계산
gid_population = population_cleaned.groupby('gid')['total_population'].sum()

# 상위 10개 지역 데이터 준비
top_10_gid = gid_population.sort_values(ascending=False).head(10).reset_index()

# Seaborn Barplot
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.barplot(data=top_10_gid, x='gid', y='total_population', palette="viridis")

# 스타일링
plt.title("Top 10 Regions by Total Population", fontsize=16)
plt.xlabel("GID", fontsize=12)
plt.ylabel("Total Population", fontsize=12)
plt.xticks(rotation=45, fontsize=10)  # GID 축 레이블 회전
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


- 특정 지역(GID)이 인구가 상대적으로 많음
- 인구가 많은 지역은 공실률이 낮을 가능성이 크며, 이는 주거 수요와 직접적인 관련이 있을 수 있음

# 2. 상권정보
- com_lc_cd    
  - 상권업종대분류코드
- com_lc_nm    
  - 상권업종대분류명
- com_mc_cd    
  - 상권업종중분류코드
- com_mc_nm    
  - 상권업종중분류명
- com_sc_cd    
  - 상권업종소분류코드
- com_sc_nm    
  - 상권업종소분류명
- ksic_cd    
  - 표준산업분류코드
- ksic_nm    
  - 표준산업분류명
- emd_cd    
  - 읍면동코드
- emd_nm    
  - 읍면동명
- road_address    
  - 업소의 도로명주소
- lon    
  - 경도
- lat    
  - 위도
- std_year    
  - 데이터기준연도

In [ ]:
market_info.head()

In [ ]:
pd.set_option('display.max_rows', None)
market_info[['com_lc_nm', 'com_mc_nm', 'com_sc_nm', 'ksic_nm']].value_counts()

In [ ]:
market_info['adb_emd_nm'].nunique()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def analyze_district_categories(df):
    """
    Analyze business categories distribution by district, including middle and small categories.

    Parameters:
    df (pandas.DataFrame): DataFrame containing business and district information
    """
    # 지역별 대분류, 중분류, 소분류 업종 수 계산
    district_category_counts = pd.crosstab(
        index=df['adb_emd_nm'],
        columns=[df['com_lc_nm'], df['com_mc_nm'], df['com_sc_nm']]
    )

    # 지역별 총 업소 수 계산
    total_by_district = district_category_counts.sum(axis=1).sort_values(ascending=False)

    # 지역별 상위 3개 대분류 찾기
    top_3_categories = {}
    for district in district_category_counts.index:
        # 대분류(`com_lc_nm`) 레벨별 합계를 계산
        district_data = district_category_counts.groupby(level=0, axis=1).sum()
        top_3 = district_data.loc[district].nlargest(3)
        top_3_categories[district] = [(cat, count) for cat, count in top_3.items()]

    # 결과 DataFrame 생성
    results = pd.DataFrame({
        '총 업소 수': total_by_district,
        '상위 3개 대분류': top_3_categories
    })

    # 지역별 중분류 및 소분류 상세 데이터 저장
    detailed_categories = {}
    for district in district_category_counts.index:
        district_data = district_category_counts.loc[district]
        top_middle_small = district_data[district_data > 0].sort_values(ascending=False).head(5)
        detailed_categories[district] = top_middle_small.to_dict()

    results['상세 업종'] = detailed_categories

    return results


def print_analysis(results):
    """
    Print the analysis results in a formatted way
    """
    print("\n=== 지역별 업종 분석 결과 ===")
    print("\n--- 지역별 총 업소 수 (상위 5개 지역) ---")
    print(results['총 업소 수'].head())

    print("\n--- 지역별 상위 3개 대분류 ---")
    for idx, row in results.iterrows():
        print(f"\n{idx}:")
        print(f"  - 총 업소 수: {row['총 업소 수']}개")
        for rank, (category, count) in enumerate(row['상위 3개 대분류'], 1):
            print(f"  - {rank}위: {category} ({count}개)")

        print("  - 상세 업종(상위 5개):")
        for (lc, mc, sc), count in row['상세 업종'].items():
            print(f"    - {lc} > {mc} > {sc}: {count}개")


# 결과 출력
results = analyze_district_categories(market_info)
print(print_analysis(results))


## 1. 지역별 주요 업종 현황
- **서현1동:**

  - 총 업소 수 11,631개로 가장 많은 업소를 보유
음식점(3,779개)과 소매업(3,708개)이 상위 업종으로, 경쟁이 매우 치열한 상업 지역
커피전문점과 한식당의 비중이 높아 외식 중심 상권임

- **구미동, 금곡동:**

  - 각각 5,032개, 4,100개 업소를 보유
음식점과 소매업이 상위 업종으로 나타나며, 주거 지역과 상업 지역이 혼합된 형태

- **백현동, 위례동:**

  - 소매업과 음식점 비중이 높음
위례동은 생활서비스 업종(미용실, 피부관리)도 높은 비중을 차지하며 주거 편의성이 높은 지역


## 2. 지역별 상권 특징
- **교육 중심 지역:**

  - 수내3동, 이매1동, 서현2동 등 학문/교육 업종 비중이 높은 지역
  - 보습학원과 외국어 학원이 상위 업종으로, 학부모와 학생을 타겟으로 한 서비스 기회 존재

- **생활 편의 중심 지역:**

  - 상대원1동, 금광1동, 중앙동 등은 생활서비스(미용실, 청소 서비스)가 상위 업종에 포함
  - 주거지 중심 지역으로 거주민을 위한 편의 시설 비중이 높음

- **상업 밀집 지역:**

  - 서현1동, 성남동은 소매업과 음식점이 다수를 차지하며, 상업 활동이 활발한 지역
  - 화장품 판매점, 의류 매장 등의 소매업 비중이 두드러짐

In [ ]:
print(market_info.duplicated().sum())  # 중복 행 개수 확인

# 3. 상가개폐업

- service_nm    
  - 개방서비스명
- lcpmt_dt    
  - 지정일자
- rtrcn_dt    
  - 인허가취소일자
- biz_stts_cd    
  - 영업상태구분코드
- biz_stts_nm    
  - 영업상태명
- dtls_stts_cd    
  - 상세영업상태코드
- dtls_stts_nm    
  - 상세영업상태명
- cls_date    
  - 폐업일자
- tc_strt_dt    
  - 휴업시작일자
- tc_end_dt    
  - 휴업종료일자
- re_op_dt    
  - 재개업일자
- plc_area    
  - 소재지면적(㎡)
- zip_cd    
  - 소재지우편번호
- addr    
  - 소재지전체주소
- rd_addr    
  - 도로명전체주소
- rd_zip_cd    
  - 도로명우편번호
- bplc_nm    
  - 사업장명
- biz_type    
  - 업태구분명
- lon    
  - 대상위치 경도
- lat    
  - 대상위치 위도

In [ ]:
market_open.head()

In [ ]:
market_open[['service_nm', 'biz_stts_nm', 'biz_type']].value_counts().head(10)


## 1. 상위 서비스 유형
- 가장 많은 건수는 **즉석판매제조가공업**으로, 폐업 상태에서 높은 빈도를 차지
- 이는 즉석판매제조가공업이 진입장벽이 낮거나 경쟁이 치열하여 폐업률이 높을 가능성을 시사

## 2. 일반음식점(한식)
- **일반음식점(한식)**은 **영업/정상 상태와 폐업 상태 모두 상위권**에 위치
- 한식당은 지역과 상관없이 높은 수요를 가지고 있지만, 동시에 경쟁이 치열하여 폐업률도 높음을 나타냄

## 3. 통신판매업
- **통신판매업은 영업/정상 상태에서 상위권을 차지**


In [ ]:
# 영업 상태별 평균 면적
mean_plc_area = market_open.groupby('biz_stts_nm')['plc_area'].mean()
print("영업 상태별 평균 면적:")
print(mean_plc_area)

# 영업 상태별 면적 분포 데이터
print("\n영업 상태별 면적 데이터:")
for status, group in market_open.groupby('biz_stts_nm'):
    print(f"\n영업 상태: {status}")
    print(group['plc_area'].describe())


## 1. 영업 상태별 평균 면적
- 영업/정상: 평균 면적이 약 **208㎡**으로, 운영 중인 시설 중에서 비교적 평균적인 크기

- 취소/말소/만료/정지/중지: 평균 면적이 약 **57㎡**로 가장 낮습니다.

  - 이는 소규모 시설이 규제, 시장 경쟁 등으로 인해 운영이 중단되었을 가능성

- 폐업: 평균 면적이 약 **80㎡**로, 소규모 시설이 주로  폐업했음을 시사


- 휴업: 평균 면적이 **1569㎡**로 다른 상태에 비해 매우 큽니다.
  - 이는 면적이 큰 시설이 운영을 잠시 멈추는 경우가 많다는 것을 의미(예: 대규모 공장, 대형 시설)

## 2. 영업 상태별 데이터 분포
- 영업/정상
  - 평균 면적은 약 **208㎡**이지만, **표준편차가 6559㎡**로 매우 큼
  - 일부 매우 큰 시설(최대 868,162㎡)이 평균을 크게 끌어올렸음
  - 중앙값(50%): **40㎡**로, 대부분의 운영 시설은 상대적으로 작습니다.

- 취소/말소/만료/정지/중지
평균 면적이 작고, **75% 이상의 시설 면적이 0㎡**입니다.
시설이 매우 작거나, 운영 중지 상태로 데이터가 부정확하게 기록되었을 가능성

- 폐업
평균 면적이 **80㎡**로, 운영 중 시설보다 작음
  - 중앙값(50%): 31㎡, 대부분의 폐업 시설은 소규모 사업장임을 나타냄
  - 일부 대규모 시설(최대 407,347㎡)도 폐업했지만, 이는 극히 드문 경우

- 휴업
평균 면적이 **1569㎡**로, 주로 대규모 시설이 포함

  - 중앙값(50%): 7.6㎡, 작은 시설과 매우 큰 시설(최대 16,792㎡) 모두 포함되어 분포가 넓음을 시사

## 3. 시사점
- 소규모 사업장 폐업 위험:

  - 평균 면적이 작을수록 폐업 위험이 크며, 중앙값을 기준으로 하면 31㎡ 이하인 시설이 특히 취약

- 대규모 시설의 휴업 경향:

  - 휴업 상태에 있는 시설의 평균 면적이 크다는 점은 대규모 시설이 운영 중단을 선택할 가능성이 있음을 시사합니다.
이는 대규모 공장, 창고 또는 공공시설 등이 포함될 가능성이 높음

- 운영 중지 시설의 작은 규모:

  - 운영 중지(취소/말소/정지) 상태의 시설 대부분이 매우 작습니다(면적이 0 또는 매우 낮음)
  - 사업의 지속 가능성이 낮거나 법적/행정적 이유로 인한 운영 중단일 가능성이 있음

- 시설 면적과 생존률 관계:

  - 운영 상태(영업/정상)에서 큰 시설의 생존 가능성이 높음을 나타내며, 이는 자본 및 자원이 풍부한 대규모 사업체의 경쟁 우위를 시사



In [ ]:
# 상관관계
correlation = market_open[['plc_area', 'lon', 'lat']].corr()
correlation

- 지역에 따른 차이 거의 없음

# 4. 성남시 개별 공시지가


- PNU    
  - 필지고유번호
- bjd_cd    
  - 법정동코드
- bjd_nm    
  - 법정동명
- jibun    
  - 지번
- year    
  - 기준연도
- month    
  - 기준월
- land_value    
  - 공시지가(원/㎡)
- st_gbn    
  - 표준지여부


In [ ]:
seperate_cost.head()

In [ ]:
seperate_cost.info()

In [ ]:
seperate_cost.describe()

In [ ]:
# 연도별 평균 공시지가
yearly_avg = seperate_cost.groupby('year')['land_value'].mean()
print(yearly_avg)

# 연도별 시각화
yearly_avg.plot(kind='line', title='연도별 평균 공시지가')
plt.ylabel('평균 공시지가 (원/㎡)')
plt.show()


In [ ]:
# 공시지가 분포 시각화 (x축 세분화)
seperate_cost['land_value'].plot(kind='hist', bins=100, title='공시지가 분포 (세분화)')
plt.xlabel('공시지가 (원/㎡)')
plt.xlim(0, seperate_cost['land_value'].quantile(0.95))  # 상위 5% 값 제외
plt.show()


- 평균 공시지가는 2020년부터 꾸준히 상승하다가 2022년에 최고점을 기록한 후 2023년에 약간 감소

In [ ]:
# 지역별 평균 공시지가
region_avg = seperate_cost.groupby('bjd_nm')['land_value'].mean().sort_values(ascending=False)
print(region_avg)

In [ ]:
# 지번별 공시지가 상위/하위 10개
print("상위 10개 공시지가:")
print(seperate_cost.nlargest(10, 'land_value')[['bjd_nm', 'jibun', 'land_value']])

print("\n하위 10개 공시지가:")
print(seperate_cost.nsmallest(10, 'land_value')[['bjd_nm', 'jibun', 'land_value']])


In [ ]:
population.head()

,gid,year,m_20g_pop,w_20g_pop,m_30g_pop,w_30g_pop,m_40g_pop,w_40g_pop,m_50g_pop,w_50g_pop,m_60g_pop,w_60g_pop,m_70g_pop,w_70g_pop,m_80g_pop,w_80g_pop,m_90g_pop,w_90g_pop,m_100g_pop,w_100g_pop
0,다사730391,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,다사676304,2023,25.0,14.0,18.0,22.0,23.0,24.0,23.0,27.0,18.0,14.0,0.0,8.0,0.0,0.0,NaN,0.0,NaN,NaN
2,다사676300,2023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,다사676301,2023,0.0,0.0,0.0,0.0,0.0,0.0,7.0,8.0,7.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,다사632298,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
market_info.head()

,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,adb_emd_cd,adb_emd_nm,road_address,lon,lat,std_year
0,D,소매,D14,운동/경기용품소매,D14A01,운동/경기용품,G47631,운동 및 경기용품 소매업,4113567000,구미동,경기도 성남시 분당구 구미로144번길 8,127.123291,37.340903,2020
1,D,소매,D03,종합소매점,D03A01,편의점,G47122,체인화 편의점,4113554500,정자동,경기도 성남시 분당구 정자로76번길 5,127.113508,37.362807,2020
2,D,소매,D05,의복의류,D05A09,남성의류전문점,NaN,NaN,4113558000,서현1동,경기도 성남시 분당구 성남대로 601,127.123421,37.385003,2020
3,Q,음식,Q01,한식,Q01A03,곱창/양구이전문,I56111,한식 음식점업,4113562000,야탑1동,경기도 성남시 분당구 장미로 86,127.130217,37.413500,2020
4,L,부동산,L03,분양,L03A02,상가분양,L68122,비주거용 건물 개발 및 공급업,4113558000,서현1동,경기도 성남시 분당구 분당로53번길 9,127.121333,37.383915,2020


In [ ]:
market_open.head(3)

,service_nm,lcpmt_dt,rtrcn_dt,biz_stts_cd,biz_stts_nm,dtls_stts_cd,dtls_stts_nm,cls_date,tc_strt_dt,tc_end_dt,re_op_dt,plc_area,zip_cd,addr,rd_addr,rd_zip_cd,bplc_nm,biz_type,lon,lat
0,민방위급수시설,2015-12-31,2024-05-21,4.0,취소/말소/만료/정지/중지,19,사용중지,2024-05-21,NaN,NaN,NaN,50.0,NaN,경기도 성남시 분당구 석운동 산 16번지 1호,NaN,NaN,석운동 산 16-1 라이프원 코리아(2호공),NaN,127.038069,37.386796
1,민방위급수시설,2015-12-31,2024-05-21,4.0,취소/말소/만료/정지/중지,19,사용중지,2024-05-21,NaN,NaN,NaN,50.0,NaN,경기도 성남시 분당구 석운동 산 16번지 1호,NaN,NaN,석운동 산16-1 라이프원코리아(1호공),NaN,127.038069,37.386796
2,대기오염물질배출시설설치사업장,2021-10-20,NaN,1.0,영업/정상,11,영업,NaN,NaN,NaN,NaN,NaN,NaN,경기도 성남시 분당구 석운동 산 16-1,NaN,NaN,한국보안정보연구원,정부기관 일반 보조 행정,127.038069,37.386796


In [ ]:
seperate_cost.head()

,PNU,bjd_cd,bjd_nm,jibun,year,month,land_value,st_gbn
0,4113110300101220000,4113110300,경기도 성남시 수정구 수진동,122,2020,1,2662000,N
1,4113110300101230000,4113110300,경기도 성남시 수정구 수진동,123,2020,1,2745000,N
2,4113110300101250000,4113110300,경기도 성남시 수정구 수진동,125,2020,1,2745000,N
3,4113110300101260000,4113110300,경기도 성남시 수정구 수진동,126,2020,1,2745000,N
4,4113110300101270000,4113110300,경기도 성남시 수정구 수진동,127,2020,1,2512000,N


In [ ]:
market_info.head(1)

,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,adb_emd_cd,adb_emd_nm,road_address,lon,lat,std_year
0,D,소매,D14,운동/경기용품소매,D14A01,운동/경기용품,G47631,운동 및 경기용품 소매업,4113567000,구미동,경기도 성남시 분당구 구미로144번길 8,127.123291,37.340903,2020


In [ ]:

# 필요한 열 추출 (위도와 경도 포함)
population = population[["gid", "year", "m_20g_pop", "w_20g_pop"]]  # 위도, 경도는 없지만 나중 사용 가능
market_info = market_info[["lon", "lat", "com_lc_nm", "com_mc_nm", 'com_sc_nm', "ksic_nm", 'adb_emd_cd', 'adb_emd_nm', 'road_address', 'std_year']]
market_open = market_open[["service_nm",	"lcpmt_dt",	"rtrcn_dt",	"biz_stts_cd",	"biz_stts_nm",	"dtls_stts_cd", "dtls_stts_nm", "cls_date", "plc_area", "addr", "bplc_nm", "lon","lat"]]

# 위도, 경도를 기준으로 병합
merged_data = market_info.merge(market_open, on=["lon", "lat"], how="outer")  # market_info와 market_open 병합


In [ ]:
market_info.shape

(157654, 10)

In [ ]:
market_open.shape

(173319, 13)

In [ ]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330973 entries, 0 to 330972
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   lon           330973 non-null  float64
 1   lat           330973 non-null  float64
 2   com_lc_nm     157654 non-null  object 
 3   com_mc_nm     157654 non-null  object 
 4   com_sc_nm     157654 non-null  object 
 5   ksic_nm       150253 non-null  object 
 6   adb_emd_cd    157654 non-null  float64
 7   adb_emd_nm    157654 non-null  object 
 8   road_address  157654 non-null  object 
 9   std_year      157654 non-null  float64
 10  service_nm    173319 non-null  object 
 11  lcpmt_dt      173319 non-null  object 
 12  rtrcn_dt      1247 non-null    object 
 13  biz_stts_cd   173319 non-null  float64
 14  biz_stts_nm   173319 non-null  object 
 15  dtls_stts_cd  173319 non-null  object 
 16  dtls_stts_nm  173229 non-null  object 
 17  cls_date      92639 non-null   object 
 18  plc_

In [ ]:
pip install folium pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.0 MB/s eta 0:00:00


In [ ]:
import folium
import pandas as pd


center_lat = merged_data["lat"].mean()
center_lon = merged_data["lon"].mean()

# 지도 객체 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# 데이터 프레임에서 위도(lat), 경도(lon) 값을 가져와서 지도에 마커 추가
for _, row in merged_data.iterrows():
    if pd.notnull(row["lat"]) and pd.notnull(row["lon"]):  # 위도, 경도 값이 있는 경우만 처리
        folium.Marker(
            location=[row["lat"], row["lon"]],
            popup=row.get("road_address", "No Address"),  # 팝업 내용: 도로명 주소
            tooltip=row.get("service_nm", "Unknown")  # 툴팁 내용: 서비스명
        ).add_to(m)

# 지도 출력
m


Buffered data was truncated after reaching the output size limit.

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# 결측치 처리 함수
def handle_missing_values(data, method="drop"):
    if method == "drop":
        # 결측치가 있는 행 제거
        return data.dropna()
    elif method == "interpolate":
        # 선형 보간법으로 결측치 처리 (수치형 열만 처리)
        numeric_data = data.select_dtypes(include=["float64", "int64"])
        interpolated_data = numeric_data.interpolate(method="linear", limit_direction="forward", axis=0)
        # 비수치형 데이터와 병합
        non_numeric_data = data.select_dtypes(exclude=["float64", "int64"])
        return pd.concat([interpolated_data, non_numeric_data], axis=1)
    elif method == "knn":
        # KNN Imputer로 결측치 처리 (수치형 열만 처리)
        numeric_data = data.select_dtypes(include=["float64", "int64"])
        knn_imputer = KNNImputer(n_neighbors=5)
        imputed_data = pd.DataFrame(knn_imputer.fit_transform(numeric_data), columns=numeric_data.columns)
        # 비수치형 데이터와 병합
        non_numeric_data = data.select_dtypes(exclude=["float64", "int64"])
        return pd.concat([imputed_data, non_numeric_data], axis=1)
    else:
        raise ValueError("Invalid missing value handling method. Choose from 'drop', 'interpolate', or 'knn'.")

# 정규화 함수
def normalize_data(data, method="standard"):
    if data.empty:
        print("Dataframe is empty after missing value handling. Skipping normalization.")
        return None

    if method == "standard":
        scaler = StandardScaler()
    elif method == "minmax":
        scaler = MinMaxScaler()
    elif method == "robust":
        scaler = RobustScaler()
    else:
        raise ValueError("Invalid normalization method. Choose from 'standard', 'minmax', or 'robust'.")

    # 수치형 데이터만 정규화
    numeric_columns = data.select_dtypes(include=["float64", "int64"]).columns
    data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
    return data

# 처리와 저장을 수행하는 함수
def process_and_save(data, missing_method, norm_method, output_prefix="output"):
    # 결측치 처리
    handled_data = handle_missing_values(data, method=missing_method)

    # 데이터가 비어 있으면 스킵
    if handled_data.empty:
        print(f"No data left after {missing_method} missing value handling. Skipping {norm_method} normalization.")
        return

    # 정규화
    normalized_data = normalize_data(handled_data, method=norm_method)
    if normalized_data is None:
        return  # 정규화 실패 시 스킵

    # 파일 저장
    filename = f"{output_prefix}_{missing_method}_{norm_method}.csv"
    normalized_data.to_csv(filename, index=False)
    print(f"Saved: {filename}")

# 전체 프로세스 실행 함수
def process_all_cases(data, output_prefix="output"):
    missing_methods = ["drop", "interpolate", "knn"]
    norm_methods = ["standard", "minmax", "robust"]

    for missing_method in missing_methods:
        for norm_method in norm_methods:
            process_and_save(data, missing_method, norm_method, output_prefix)

# 데이터 처리 실행
process_all_cases(merged_data, output_prefix="merged_data")


No data left after drop missing value handling. Skipping standard normalization.
No data left after drop missing value handling. Skipping minmax normalization.
No data left after drop missing value handling. Skipping robust normalization.
Saved: merged_data_interpolate_standard.csv
Saved: merged_data_interpolate_minmax.csv
Saved: merged_data_interpolate_robust.csv
Saved: merged_data_knn_standard.csv
